## TODO

- [ ] load data
- [ ] divide data to train/test/val
- [ ] create object of pipeline:
    - [ ] create class of pipeline:
        - [ ] must contain fit and predict methods
        - [ ] must skip steps which don't need during inference
        - [ ] must contain trained model
- [ ] train model
- [ ] evaluate model

## Imports

In [2]:
import pandas as pd
from sklearn.model_selection import cross_val_score, TimeSeriesSplit

from Pipeline import Pipeline, GroupAndSummarize

## Data loading

In [13]:
train: pd.DataFrame = pd.read_csv('data/sales_post_process.csv')
X = [train[train["cluster"] == i] for i in range(int(train["cluster"].max()))]
train

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,shop_name,item_name,item_category_id,item_category_name,cluster
0,2013-01-01,0,42,11249,398.46,2.0,"СПб ТК ""Невский Центр""",ДУХLESS,40.0,Кино - DVD,7.0
1,2013-01-01,0,19,5572,1590.00,1.0,"Курск ТЦ ""Пушкинский""",PS Move Motion Controller (Контроллер движений...,2.0,Аксессуары - PS3,7.0
2,2013-01-01,0,54,21688,499.00,1.0,"Химки ТЦ ""Мега""",ЧЕРНАЯ ДЫРА (BD),37.0,Кино - Blu-Ray,7.0
3,2013-01-01,0,54,4779,499.00,1.0,"Химки ТЦ ""Мега""","Mass Effect 3 [PC, русские субтитры]",30.0,Игры PC - Стандартные издания,7.0
4,2013-01-01,0,46,22134,399.00,2.0,"Сергиев Посад ТЦ ""7Я""","Я, АЛЕКС КРОСС",40.0,Кино - DVD,7.0
...,...,...,...,...,...,...,...,...,...,...,...
12978,2013-01-03,0,42,2808,999.00,1.0,"СПб ТК ""Невский Центр""","Diablo III [PC, Jewel, русская версия]",30.0,Игры PC - Стандартные издания,7.0
12979,2013-01-03,0,6,2808,999.00,5.0,"Воронеж (Плехановская, 13)","Diablo III [PC, Jewel, русская версия]",30.0,Игры PC - Стандартные издания,0.0
12980,2013-01-03,0,50,6240,299.00,1.0,"Тюмень ТЦ ""Гудвин""",SCORPIONS Comeblack,55.0,Музыка - CD локального производства,2.0
12981,2013-01-03,0,59,12805,399.00,1.0,"Ярославль ТЦ ""Альтаир""",КОРОЛЬ ЛЕВ (регион),40.0,Кино - DVD,1.0


## Train/Validation Pipeline

In [11]:
tscv = TimeSeriesSplit(n_splits=4)
pipeline: Pipeline = Pipeline([
    ("group", GroupAndSummarize(group_by_col=["date"], agg_dict={"item_cnt_day": "sum"}), False),
])

for cluster in X:
    x = pipeline.transform(cluster)
    print(cluster["date"].max())
    for train_index, test_index in tscv.split(x):
        X_train, X_test = cluster[train_index], cluster[test_index]
        print(X_train, X_test)

2013-01-03


ValueError: Cannot have number of folds=5 greater than the number of samples=3.

## Preprocessing steps